In [1]:
import requests
import json
from json2df import flatten_dict
import pandas as pd
from lxml import etree

In [4]:
fd = '../download_soyoung/15W_htmls_diary/7085323.html'

In [229]:
class Hparser(object):
    collection = []
    df=None
    N=0
    def __init__(self, html):
        self.coll = []
        self.diaryID = html.split('/')[-1].split('.html')[0]
        try:
            with open(html) as fh:self.tree = etree.HTML(fh.read())
        except Exception as e:
            print e, 'X0'
            print diaryID        
        print self.diaryID, self.tree
        
        
        self.main()
        
        
    @classmethod
    def _update(cls, new):
        cls.N += 1
        cls.collection+=new
        
    def _product(self):
        try: self.pid = self.tree.xpath('//div[@class="value product-name"]/a/@href')[0].split('/')[-1]
        except: print 'X pid'
    def _content(self):
        try:self.content = self.tree.xpath('//li[@class="diary-item"]'); self.dn = len(self.content)
        except: print 'X content'
            
    def _image(self, node):
        dic={}
        dic['diary_id']=self.diaryID
        dic['product_id']=self.pid
        
        dic['type']='image_post'
        dic['title']=node[0].xpath('text()')[0]
        dic['text']=node[1][0].xpath('text()')[0]
        dic['post_id']=node[1][0].xpath('./@href')[0].split('?')[0].split('/')[-1]
        for n, i in enumerate(node[2]):
            dic['image_'+str(n+1)] = i[0][0].xpath('./@data-img')[0]
        n=0
        for i in node[3]:
            if i.tag=='div': dic['time']=i.xpath('text()')[0]
            else:
                if len(i)!=0:
                    n+=1
                    if n==1:
                        dic['views']=i[0].xpath('text()')[0]
                    elif n==2:
                        dic['comments']=i[0].xpath('text()')[0]
                    else:
                        dic['favor']=i[0].xpath('text()')[0]
        return dic        
    def _video(self, node):
        dic={}
        dic['diary_id']=self.diaryID
        dic['product_id']=self.pid
        dic['type']='video_post'
        dic['text']=node[1][0].xpath('text()')[0]
        dic['title']=node[0].xpath('text()')[0]
        dic['post_id']=node[1][0].xpath('./@href')[0].split('?')[0].split('/')[-1]
        
        dic['video_link']=node[2][0].xpath('./@href')[0]
        dic['video_image']=node[2][0].xpath('./img')[0].xpath('./@data-img')[0]   
        
        n=0
        for i in node[3]:
            if i.tag=='div': dic['time']=i.xpath('text()')[0]
            else:
                if len(i)!=0:
                    n+=1
                    if n==1:
                        dic['views']=i[0].xpath('text()')[0]
                    elif n==2:
                        dic['comments']=i[0].xpath('text()')[0]
                    else:
                        dic['favor']=i[0].xpath('text()')[0]        
        return dic
    
    def loop(self):
        for p in self.content:
            
            if p[2].tag == 'ul':
                self.coll.append(self._image(p))
            else:
                self.coll.append(self._video(p))
                
    def main(self):
        self._product()
        self._content()
        self.loop()
        self._update(self.coll)
        
    def __repr__(self):
        return "diary: {} of product: {} has {} posts, all {} posts finished parsing"\
    .format(self.diaryID, self.pid, self.dn, self.N)

In [230]:
fd

'../download_soyoung/15W_htmls_diary/7085323.html'

In [231]:
x = Hparser(fd)
x

7085323 <Element html at 0x2aff92593440>


diary: 7085323 of product: cp55737 has 2 posts, all 1 posts finished parsing

In [232]:
x.N

1

In [234]:
# x.coll

In [239]:
pd.DataFrame(Hparser.collection)

,comments,diary_id,favor,image_1,image_2,image_3,image_4,image_5,image_6,image_7,...,image_9,post_id,product_id,text,time,title,type,video_image,video_link,views
0,1,7085323,0,http://img2.soyoung.com/tieba/web/20171225/5/2...,http://img2.soyoung.com/tieba/web/20171225/9/2...,http://img2.soyoung.com/tieba/web/20171225/8/2...,http://img2.soyoung.com/tieba/web/20171225/8/2...,http://img2.soyoung.com/tieba/web/20171225/0/2...,http://img2.soyoung.com/tieba/web/20171225/7/2...,http://img2.soyoung.com/tieba/web/20171225/0/2...,...,http://img2.soyoung.com/tieba/web/20171225/7/2...,p16211010,cp55737,【玻尿酸丰苹果肌第40天】圣诞节快乐，男神女神们，昨天晚上你们是怎么度过的？为什么我袜子都准...,2017-12-25 18:01:47,术后第40天,image_post,NaN,NaN,15
1,2,7085323,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,p16125464,cp55737,【玻尿酸丰苹果肌第35天】特意撸了一个小视频，哈哈哈，让大家欣赏一下！美无止境，适可而止。变...,2017-12-20 16:23:02,术后第35天,video_post,http://img2.soyoung.com/message/ios/20171220/9...,/p16125464?group_id=7085323#VideoAnchor,23


In [238]:
# Hparser.collection

In [197]:
t

<Element li at 0x2aff9259d128>

In [117]:
'a b'.split(' ')

['a', 'b']

In [118]:
t[3].xpath('div/text()')

['2017-12-25 18:01:47']

In [145]:
for i in t[3]:
    print i.tag,i.xpath('text()')
    if len(i)!=0:print i[0].xpath('text()'), i[0]

div ['2017-12-25 18:01:47']
span []
span []
['15'] <Element a at 0x2aff923907a0>
span []
span []
['1'] <Element a at 0x2aff92390dd0>
span []
span []
['0'] <Element a at 0x2aff92390f38>


In [120]:
# a = requests.get(link)
c = etree.HTML(text)
c

<Element html at 0x2aff9258c170>

In [207]:
for i in d[2][0]:
    print i

<!-- <img src="//static.soyoung.com/pc/static/img/dp/before.jpg" alt=""> -->
<Element img at 0x2aff92b2cc20>
<Element span at 0x2aff92b2ca28>


### map product

In [67]:
i = c.xpath('//div[@class="value product-name"]/a/@href')[0]
i

'http://y.soyoung.com/cp55737'

In [57]:
i[0].items()

[('href', 'http://y.soyoung.com/cp55737')]

In [7]:
print i[0].text

【伊婉玻尿酸1ml】韩国进口玻尿酸 丰下巴头丰眉弓丰苹果肌隆鼻泪沟法令纹 大分子玻尿酸


### find posts

In [8]:
# //*[@id="mengceng-box"]/div[2]/ul/li[2]

In [75]:
j = c.xpath('//li[@class="diary-item"]')

In [76]:
len(j)

2

In [198]:
t, d = j[0], j[1]

In [199]:
t.tag

'li'

In [96]:
t[2][0][0][0].xpath('./@data-img')

['http://img2.soyoung.com/tieba/web/20171225/5/20171225175922590_152_152.jpg']

In [12]:
print dir(t)

['__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__doc__', '__format__', '__getattribute__', '__getitem__', '__hash__', '__init__', '__iter__', '__len__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '_init', 'addnext', 'addprevious', 'append', 'attrib', 'base', 'clear', 'cssselect', 'extend', 'find', 'findall', 'findtext', 'get', 'getchildren', 'getiterator', 'getnext', 'getparent', 'getprevious', 'getroottree', 'index', 'insert', 'items', 'iter', 'iterancestors', 'iterchildren', 'iterdescendants', 'iterfind', 'itersiblings', 'itertext', 'keys', 'makeelement', 'nsmap', 'prefix', 'remove', 'replace', 'set', 'sourceline', 'tag', 'tail', 'text', 'values', 'xpath']


https://www.w3schools.com/xml/xpath_syntax.asp

In [13]:
try:
    print t.tag
    print t[0].tag, t[0].text
    print t[1].tag, t[1][0].tag, t[1][0].xpath('./@href')[0], t[1][0].items()[-1][-1],t[1][0].text
    
    print t[2].tag
    for g in t[2]:
        print g[0].tag, g[0].items()[-1][-1], g[0][0].tag, g[0][0].items()[2][-1]
    print t[3].tag, len(t[3])
    
    for x in t[3]: 
        if x.tag=='div':print x.text
        else: 
            if len(x)==0:print x.xpath('./@class')[0]
            else:print x[0].text
except:
    print 'X'

li
span 术后第40天
p a /p16211010?group_id=7085323 /p16211010?group_id=7085323 【玻尿酸丰苹果肌第40天】圣诞节快乐，男神女神们，昨天晚上你们是怎么度过的？为什么我袜子都准备好了，今早还是没见到圣诞老爷爷送我的礼物噢月到年底越努力越幸运，工作的时候开开心心的工作，玩的时候开开心心的玩。昨天是平安夜，特意与
ul
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/5/20171225175922590_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/9/20171225175923870_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/8/20171225175923197_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/8/20171225175924432_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/0/20171225175924134_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/7/20171225175925273_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/0/20171225175927736_152_152.jpg
a /p16211010?group_id=7085323 img http://img2

In [14]:
for i in t:print i

<Element span at 0x2b0125d02998>
<Element p at 0x2b0125d027a0>
<Element ul at 0x2b0125591b90>
<Element div at 0x2b0125d02998>


In [15]:
# for l in t:
#     a = l.xpath('./span[@class="day"]')
#     b = l.xpath('./p[@class="describe"]')
#     print a
#     print b
#     print 

In [16]:
a = [1,2,3,4]

In [17]:
a.remove(0)

ValueError: list.remove(x): x not in list

In [18]:
ID = 7085323

link = "http://www.soyoung.com/dpg{}".format(ID)

a = requests.get(link)
c = etree.HTML(a.text)

i = c.xpath('//div[@class="value product-name"]/a')
print i[0].items()[-1][-1]
print i[0].text

j = c.xpath('//li[@class="diary-item"]')
_, t = j[0], j[1]

http://y.soyoung.com/cp55737
【伊婉玻尿酸1ml】韩国进口玻尿酸 丰下巴头丰眉弓丰苹果肌隆鼻泪沟法令纹 大分子玻尿酸


In [19]:
# //*[@id="mengceng-box"]/div[2]/ul/li[2]/p/a/text()

### video

In [20]:
try:
    print t.tag
    print t[0].tag, t[0].text
    print t[1].tag, t[1][0].tag, t[1][0].xpath('./@href')[0], t[1][0].items()[-1][-1]
    print t[1][0].xpath('./text()')[0]
    
    print t[2].tag
    if t[2].tag=='ul':
        for g in t[2]:
            print g[0].tag, g[0].items()[-1][-1], g[0][0].tag, g[0][0].items()[2][-1]
        print t[3].tag, len(t[3])
    else:
        print len(t[2])
        print t[2].xpath('./@class')[0]
        print t[2][0].xpath('./@href')[0]
        print t[2][0].xpath('./img')[0].xpath('./@data-img')[0]
        
    print t[3].tag, len(t[3])
    
    print t[3]

    for x in t[3]: 
        if x.tag=='div':print x.text
        else: 
            if len(x)==0:print x.xpath('./@class')[0]
            else:print x[0].text
except Exception as e:
    print e
    print 'X'

li
span 术后第35天
p a /p16125464?group_id=7085323 /p16125464?group_id=7085323
【玻尿酸丰苹果肌第35天】特意撸了一个小视频，哈哈哈，让大家欣赏一下！美无止境，适可而止。变美丽了也要变忒别哦。因为喜欢Jay所以喜欢他那首《可爱女人》，祝愿大家都能够变美变漂亮变成男神喜欢的可爱女人噢！[:66]
div
1
video-poster
/p16125464?group_id=7085323#VideoAnchor
http://img2.soyoung.com/message/ios/20171220/9/20171220162148367_600_300.jpg
div 7
<Element div at 0x2b01260662d8>
2017-12-20 16:23:02
icon views
23
icon comments
2
icon favor
1


In [21]:
t=_

In [22]:
try:
    print t.tag
    print t[0].tag, t[0].text
    print t[1].tag, t[1][0].tag, t[1][0].xpath('./@href')[0], t[1][0].items()[-1][-1]
    print t[1][0].xpath('./text()')[0]
    
    print t[2].tag
    if t[2].tag=='ul':
        for g in t[2]:
            print g[0].tag, g[0].items()[-1][-1], g[0][0].tag, g[0][0].items()[2][-1]
        print t[3].tag, len(t[3])
    
    for x in t[3]: 
        if x.tag=='div':print x.text
        else: 
            if len(x)==0:print x.xpath('./@class')[0]
            else:print x[0].text
except:
    print 'X'

li
span 术后第40天
p a /p16211010?group_id=7085323 /p16211010?group_id=7085323
【玻尿酸丰苹果肌第40天】圣诞节快乐，男神女神们，昨天晚上你们是怎么度过的？为什么我袜子都准备好了，今早还是没见到圣诞老爷爷送我的礼物噢月到年底越努力越幸运，工作的时候开开心心的工作，玩的时候开开心心的玩。昨天是平安夜，特意与
ul
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/5/20171225175922590_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/9/20171225175923870_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/8/20171225175923197_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/8/20171225175924432_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/0/20171225175924134_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/7/20171225175925273_152_152.jpg
a /p16211010?group_id=7085323 img http://img2.soyoung.com/tieba/web/20171225/0/20171225175927736_152_152.jpg
a /p16211010?group_id=7085323 img http://img2